### Pydantic output parser

In [2]:
# from config.api_keys import openai_key
from dotenv import load_dotenv
load_dotenv()


True

In [3]:
from langchain_core.output_parsers import PydanticOutputParser
from langchain_core.prompts import PromptTemplate
from langchain_openai import ChatOpenAI
from pydantic import BaseModel, Field

class MovieInfo(BaseModel):
    title: str = Field(description="영화 제목")  # LLM이 description을 반영
    director: str = Field(description="영화 감독")
    release_year: int = Field(description="개봉 연도")
    
parser = PydanticOutputParser(pydantic_object=MovieInfo)
llm = ChatOpenAI(
    model_name='gpt-4o-mini',
    # api_key=openai_key,  # load_dotenv로 API키 적용된 상태
    temperature=0
)
prompt = PromptTemplate.from_template(
    """
    사용자가 입력한 영화의 정보를 알려주세요
    영화의 제목, 감독, 개봉년도를 알려주세요
    영화 이름: {movie_name}
    출력:{format}
    """
)
# partial varible: 사용자가 아닌 개발자가 접근하는 변수
prompt = prompt.partial(format=parser.get_format_instructions())


In [4]:
chain = (
    prompt | llm | parser
)

output = chain.invoke("극한직업")
print(output)

title='극한직업' director='이병헌' release_year=2019


In [5]:
output.json()

C:\Users\user\AppData\Local\Temp\ipykernel_51428\3742157523.py:1: PydanticDeprecatedSince20: The `json` method is deprecated; use `model_dump_json` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.10/migration/
  output.json()


'{"title":"극한직업","director":"이병헌","release_year":2019}'

### comma separated list output parser

In [8]:
from langchain_core.output_parsers import CommaSeparatedListOutputParser

parser = CommaSeparatedListOutputParser()

llm_response = '사과,딸기,바나나, 당근'

parsed_output = parser.parse(llm_response)
parsed_output



['사과', '딸기', '바나나', '당근']

In [13]:
from langchain_core.output_parsers import CommaSeparatedListOutputParser
from langchain_core.prompts import PromptTemplate
from langchain_openai import ChatOpenAI
from pydantic import BaseModel, Field

parser = CommaSeparatedListOutputParser()
llm = ChatOpenAI(
    model_name='gpt-4o-mini',
    # api_key=openai_key,  # load_dotenv로 API키 적용된 상태
    # temperature=0
)
prompt = PromptTemplate.from_template(
    '다음 주제와 관련된 3가지 항목을 콤마(,)로 구분하여 나열하세요. 주제: {topic}'
)

chain = prompt | llm | parser

output = chain.invoke('과일')
print(output)

output = chain.invoke('한식')
print(output)

output = chain.invoke('한식 메뉴')
print(output)

['사과', '바나나', '오렌지']
['김치', '비빔밥', '불고기']
['비빔밥', '불고기', '김치찌개']


### structured output parser
- pydantic output parser의 간단한 버전
- data type을 설정하지 않음

In [15]:
from langchain.output_parsers import ResponseSchema, StructuredOutputParser
from langchain_core.prompts import PromptTemplate
from langchain_openai import ChatOpenAI

schemas = [
    ResponseSchema(name='title', description='영화 제목'),
    ResponseSchema(name='director', description='영화 감독'),
    ResponseSchema(name='release_year', description='개봉 연도'),
]

parser = StructuredOutputParser.from_response_schemas(schemas)

llm = ChatOpenAI(
    model_name='gpt-4o-mini',
    # api_key=openai_key,  # load_dotenv로 API키 적용된 상태
    # temperature=0
)

prompt = PromptTemplate.from_template(
    """
    사용자가 입력한 영화의 정보를 알려주세요
    영화의 제목, 감독, 개봉년도를 알려주세요
    영화 이름: {movie_name}
    출력:{format}
    """
)
# partial varible: 사용자가 아닌 개발자가 접근하는 변수
prompt = prompt.partial(format=parser.get_format_instructions())

chain = (
    prompt | llm | parser
)

output = chain.invoke("memento")
print(output)


{'title': 'Memento', 'director': 'Christopher Nolan', 'release_year': '2000'}


In [20]:
### 책의 정보를 알려주는 인공지능 에이전트를 만들어 보세요

from langchain_core.output_parsers import PydanticOutputParser
from langchain_core.prompts import PromptTemplate
from langchain_openai import ChatOpenAI
from pydantic import BaseModel, Field

class BookInfo(BaseModel):
    title: str = Field(description="도서 제목")  # LLM이 description을 반영
    author: str = Field(description="글쓴이")
    publication_year: int = Field(description="출판 연도")
    summary: str = Field(description="내용 요약")

parser = PydanticOutputParser(pydantic_object=BookInfo)
llm = ChatOpenAI(
    model_name='gpt-4o-mini',
    # api_key=openai_key,  # load_dotenv로 API키 적용된 상태
    temperature=0
)
prompt = PromptTemplate.from_template(
    """
    사용자가 입력한 도서의 정보를 알려주세요
    도서의 제목, 글쓴이, 출판 연도를 알려주세요
    도서의 간단한 내용을 200자 이내로 요약하세요
    도서 이름: {book_name}
    출력:{format}
    """
)
# partial varible: 사용자가 아닌 개발자가 접근하는 변수
prompt = prompt.partial(format=parser.get_format_instructions())

chain = (
    prompt | llm | parser
)

output = chain.invoke("이기적 유전자")
print(output)

output = chain.invoke("해리포터")
print(output)

print(type(output))


title='이기적 유전자' author='리처드 도킨스' publication_year=1976 summary="이 책은 유전자의 관점에서 진화를 설명하며, 생물의 행동과 사회적 구조를 유전자의 생존과 복제를 중심으로 분석합니다. 도킨스는 '이기적 유전자'라는 개념을 통해 생물의 행동을 이해하는 새로운 시각을 제시합니다."
title='해리포터' author='J.K. 롤링' publication_year=1997 summary='해리포터는 마법사 학교인 호그와트에 입학한 소년 해리 포터의 모험을 다룬 이야기로, 친구들과 함께 어둠의 마법사 볼드모트를 물리치기 위해 성장하는 과정을 그린 판타지 소설입니다.'
<class '__main__.BookInfo'>


In [4]:
### 책의 정보를 알려주는 인공지능 에이전트를 만들어 보세요

from langchain_core.output_parsers import PydanticOutputParser
from langchain_core.prompts import PromptTemplate
from langchain_openai import ChatOpenAI
from pydantic import BaseModel, Field

class BookInfo(BaseModel):
    title: str = Field(description="도서 제목")  # LLM이 description을 반영
    author: str = Field(description="글쓴이")
    publication_year: int = Field(description="출판 연도")
    summary: str = Field(description="내용 요약")

parser = PydanticOutputParser(pydantic_object=BookInfo)
llm = ChatOpenAI(
    model_name='gpt-4o-mini',
    # api_key=openai_key,  # load_dotenv로 API키 적용된 상태
    temperature=0
)
prompt = PromptTemplate.from_template(
    """
    사용자가 입력한 도서의 정보를 알려주세요
    도서의 제목, 글쓴이, 출판 연도를 알려주세요
    도서의 간단한 내용을 200자 이내로 요약하세요
    도서 이름: {book_name}
    출력:{format}
    """
)
# partial varible: 사용자가 아닌 개발자가 접근하는 변수
prompt = prompt.partial(format=parser.get_format_instructions())

chain = (
    prompt | llm | parser
)

output = chain.invoke("이기적 유전자")
print(output)

output = chain.invoke("해리포터")
print(output)

print(prompt)

title='이기적 유전자' author='리처드 도킨스' publication_year=1976 summary='이 책은 유전자의 관점에서 진화론을 설명하며, 생물의 행동과 사회적 구조가 유전자에 의해 어떻게 형성되는지를 탐구합니다. 도킨스는 유전자가 생물의 진화에서 중심적인 역할을 한다고 주장합니다.'
title='해리포터' author='J.K. 롤링' publication_year=1997 summary='해리포터는 마법사 학교에 다니는 소년 해리 포터의 모험을 그린 이야기로, 친구들과 함께 어둠의 마법사와 싸우며 성장하는 과정을 담고 있습니다.'
input_variables=['book_name'] input_types={} partial_variables={'format': 'The output should be formatted as a JSON instance that conforms to the JSON schema below.\n\nAs an example, for the schema {"properties": {"foo": {"title": "Foo", "description": "a list of strings", "type": "array", "items": {"type": "string"}}}, "required": ["foo"]}\nthe object {"foo": ["bar", "baz"]} is a well-formatted instance of the schema. The object {"properties": {"foo": ["bar", "baz"]}} is not well-formatted.\n\nHere is the output schema:\n```\n{"properties": {"title": {"description": "도서 제목", "title": "Title", "type": "string"}, "author": {"description": "글쓴이", "title": "Author", "type": "strin

### JSON output parser

In [23]:
from langchain_core.output_parsers import JsonOutputParser

parser = JsonOutputParser()

llm_response = '{"title":"이기적 유전자", "author":"리처드 도킨스"}'

output = parser.parse(llm_response)
print(output)
print(type(output))


{'title': '이기적 유전자', 'author': '리처드 도킨스'}
<class 'dict'>


### 연습문제
다음과 같은 LangChain Expression Language (LCEL) 체인을 작성하세요.
사용자가 책 제목을 입력하면, LLM이 해당 책의 정보를 반환하고, 이를 PydanticOutputParser를 통해 구조화된 데이터로 변환해야 합니다.
user_input = "죄와 벌"

output = {
  "title": "죄와 벌",
  "author": "표도르 도스토옙스키",
  "publication_year": 1866,
  "isbn": "978-89-374-2172-9",
  "genre": "고전 문학"
}


In [32]:
from langchain_core.output_parsers import PydanticOutputParser
from langchain_core.prompts import PromptTemplate
from langchain_openai import ChatOpenAI
from pydantic import BaseModel, Field

class BookInfo(BaseModel):
    title: str = Field(description="도서 제목")  # LLM이 description을 반영
    author: str = Field(description="글쓴이")
    publication_year: int = Field(description="출판 연도")
    isbn: str = Field(description="ISBN 코드")
    genre: str = Field(description="도서 장르")

parser = PydanticOutputParser(pydantic_object=BookInfo)
llm = ChatOpenAI(
    model_name='gpt-4o-mini',
    # api_key=openai_key,  # load_dotenv로 API키 적용된 상태
    temperature=0
)
prompt = PromptTemplate.from_template(
    """
    사용자가 입력한 도서의 정보를 알려주세요
    도서의 제목, 글쓴이, 출판 연도, ISBN 코드, 장르를 알려주세요
    도서 이름: {book_name}
    출력:{format}
    """
)
# partial varible: 사용자가 아닌 개발자가 접근하는 변수
prompt = prompt.partial(format=parser.get_format_instructions())

chain = (
    prompt | llm | parser
)

output = chain.invoke("죄와 벌")
output

BookInfo(title='죄와 벌', author='표도르 미하일로비치 도스토예프스키', publication_year=1866, isbn='978-89-01-00000-0', genre='소설')

### 연습문제
고객 피드백 요약 (고객 지원 팀)
문한 SaaS(소프트웨어 서비스) 회사에서 고객 지원 팀이 고객 피드백을 분석하여 주요 키워드를 추출하는 시스템을 만들고 있습니다.
GPT 모델이 고객 리뷰를 분석하여 중요한 키워드를 콤마로 구분된 리스트로 출력하면,
이를 CommaSeparatedListOutputParser 를 사용하여 변환하는 코드를 작성하세요.

user_input = "이 소프트웨어는 사용이 매우 직관적이며, 빠른 로딩 속도가 마음에 듭니다. 하지만 가
격이 조금 비싼 것 같아요."

output = ["직관적", "빠른 속도", "사용자 친화적", "비싼 가격"]

In [ ]:
from langchain_core.output_parsers import CommaSeparatedListOutputParser
from langchain_core.prompts import PromptTemplate, FewShotPromptTemplate
from langchain_openai import ChatOpenAI

examples = [
        {
        'feeedback': '이 소프트웨어는 사용이 매우 주관적이며, 느린 로딩 속도가 별로입니다. 하지만 사용하기 쉽고 가격이 싸서 좋아요.',
        'keyword': '주관적, 느린 속도, 쉬운 사용성, 싼 가격'
    }
]
# prompt = PromptTemplate.from_template(
#     '고객 피드백을 분석하여 장점 및 단점에 대한 주요 키워드를 추출하세요.\n'
#     '추출한 키워드를 콤마(,)로 구분하여 나열하세요.\n고객 피드백: {feedback}'
# )
example_prompt = PromptTemplate(
    input_variables=["feeedback", "keyword"],
    template="입력: {feeedback}\n출력: {keyword}"
)

few_shot_prompt = FewShotPromptTemplate(
    examples=examples,  # 위에서 정의한 예제 데이터
    example_prompt=example_prompt,  # 예제를 넣을 템플릿
    prefix="고객 피드백을 분석하여 장점 및 단점에 대한 주요 키워드를 추출하세요.:\n",  # 프롬프트의 시작 부분
    suffix="입력: {feeedback}\n출력:",  # 사용자 입력이 들어갈 부분
    input_variables=["feeedback"]  # 동적으로 입력될 변수
)

llm = ChatOpenAI(
    model_name='gpt-4o-mini',
)

parser = CommaSeparatedListOutputParser()
chain = few_shot_prompt | llm | parser

output = chain.invoke('이 소프트웨어는 사용이 매우 직관적이며, 빠른 로딩 속도가 마음에 듭니다. 하지만 가격이 조금 비싼 것 같아요.')
print(output)


['직관적', '빠른 속도', '비싼 가격']
